---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [57]:
#Import the required libraries.
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import cross_validation
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from datetime import datetime
from dateutil import relativedelta
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier 

In [58]:
#Import the dataframes.
blight_df_train = pd.read_csv('train.csv', engine='python')
blight_df_test = pd.read_csv('test.csv', engine='python')

In [3]:
#Avoiding null values in the response label.

#Counting the proportion of nan values in the df.
y = blight_df_train['compliance']
y.columns = ['compliance']

#Calculate how many rows in the target variable are nan.
nan_percentage = y.isnull().sum()/y.shape[0]

#Erease rows having nan values in target.
index_nan = y.index[y.apply(np.isnan)].tolist()
y = y.drop(y.index[[index_nan]])
y = y.reset_index(drop=True)

#Assign to X matrixes the values obtained from the datasets.
blight_df_train = blight_df_train.drop(blight_df_train.index[[index_nan]])
X = blight_df_train
X = X.reset_index(drop=True)
features = ['ticket_id','agency_name','inspector_name','violator_name','violation_street_number','violation_street_name','violation_zip_code','mailing_address_str_number','mailing_address_str_name','city','state','zip_code','non_us_str_code','country','ticket_issued_date','hearing_date' ,'violation_code','violation_description','disposition','fine_amount','admin_fee','state_fee','late_fee','discount_amount','clean_up_cost','judgment_amount','grafitti_status']  
X = X[features]

X_test = blight_df_test
X_test = X_test[features]

print(y.shape)
print(X.shape)
print(X_test.shape)
print("")
print(X_test['ticket_id'].head(15))

(159880,)
(159880, 27)
(61001, 27)

0     284932
1     285362
2     285361
3     285338
4     285346
5     285345
6     285347
7     285342
8     285530
9     284989
10    285344
11    285343
12    285340
13    285341
14    285349
Name: ticket_id, dtype: int64


<h3>1. Test set adjustment</h3>
<p>Guaranteeing the test set meet all the requirements for the evaluation phase.</p>

In [18]:
#Define a method that indicates if a column contains NaN values.
def column_contains_na(df, df_column_name):
    b = df[df_column_name]
    contains_nan = b.isnull().values.any()
    return contains_nan

b = column_contains_na(X_test,'violation_code')
print(b)

False


In [5]:
#Define a method that returns the most frequent value of a column given by argument.
def column_most_frequent_value(df , df_colum_name):
    column = df[df_colum_name]
    most_frequent_value = column.mode().values
    most_frequent_value = most_frequent_value[0,]
    return most_frequent_value

#c = column_most_frequent_value(X_test,'violation_zip_code')
#print(c)

In [6]:
#Define a method that for a column of type date, replace NaN by an ancient date.
def column_na_date_replace_by_ancient_date(df, df_colum_name, replacement_value):
    
    df_updated = df
    if(isinstance(df_updated, pd.Series)):
        df_updated = pd.DataFrame(df_updated)
        df_updated.columns = [df_colum_name]
    b = df_updated[df_colum_name]
    b_updated = b.fillna(value=replacement_value)
    b_updated = pd.DataFrame(b_updated)
    df_updated = df_updated.drop(df_colum_name,axis=1)
    df_updated = pd.concat([df_updated,b_updated],axis=1)
    
    return df_updated

#b = X_test['hearing_date']
#b_updated = column_na_date_replace_by_ancient_date(X_test,'hearing_date','2012-01-19 09:00:00')
#contains_na = column_contains_na(b_updated,'hearing_date')
#print("")
#print(contains_na)
#print(b_updated.head(5))

In [7]:
#Define a method that for a column of type object, replace NaN by the most frequent column value.
def column_na_object_replace_by_frequent(df, df_colum_name):
    df_updated = df
    replacement_value = column_most_frequent_value(df_updated,df_colum_name)
    if(isinstance(df_updated, pd.Series)):
        df_updated = pd.DataFrame(df_updated)
        df_updated.columns = [df_colum_name]
        
    b = df_updated[df_colum_name]
    b_updated = b.fillna(value=replacement_value)
    b_updated = pd.DataFrame(b_updated)
    df_updated = df_updated.drop(df_colum_name,axis=1)
    df_updated = pd.concat([df_updated,b_updated],axis=1)
    
    return df_updated

#b = column_na_object_replace_by_frequent(X_test, 'violation_zip_code')
#c = column_contains_na(b,'violation_zip_code')
#print(c)

In [8]:
#Define a method that for a column of type object, replace NaN by the most frequent column value.
def column_na_float_replace_by_zeros(df, df_colum_name):
    df_updated = df
    if(isinstance(df_updated, pd.Series)):
        df_updated = pd.DataFrame(df_updated)
        df_updated.columns = [df_colum_name]
        
    b = df_updated[df_colum_name]
    b_updated = b.fillna(0)
    b_updated = pd.DataFrame(b_updated)
    df_updated = df_updated.drop(df_colum_name,axis=1)
    df_updated = pd.concat([df_updated,b_updated],axis=1)
    
    return df_updated

#b = column_na_float_replace_by_zeros(X_test, 'non_us_str_code')
#print(b)
#c = column_contains_na(b,'non_us_str_code')
#print(c)

In [9]:
#Define a methods that returns true if a column is of type object.
def column_is_object(df , df_column_name):
    column = df[df_column_name]
    if (column.dtype =="object"):
        return True
    else:
        return False
        
#condition = column_is_object(X_test, 'violator_name')
#print(condition)
#print(X_test.dtypes)

In [10]:
#Define a method that combines all of the adjustments in order to modify the X_test.
def adjust_X_test(df):
    
    df_analysis = df
    df_columns = df.columns.tolist()
    
    for i in df_columns:
        
        #the column contains nan values?
        if(column_contains_na(df_analysis,i)==True):
            
            #The column is object?
            if(column_is_object(df_analysis,i)==True):
                #The column contains dates as strings?
                if (i=='hearing_date')or(i=='ticket_issued_date'):
                    df_analysis = column_na_date_replace_by_ancient_date(df_analysis,i,'2020-01-19 09:00:00')
                else:
                    df_analysis = column_na_object_replace_by_frequent(df_analysis, i)
            else:
                df_analysis = column_na_float_replace_by_zeros(df_analysis,i)
           
    return df_analysis

number_of_nulls = X_test.isnull().sum().sum()
print(number_of_nulls)

X_test = adjust_X_test(X_test)
number_of_nulls = X_test.isnull().sum().sum()
print(number_of_nulls)

160335
0


In [11]:
print(X_test.shape)
print(X_test['ticket_id'].head(10))

(61001, 27)
0    284932
1    285362
2    285361
3    285338
4    285346
5    285345
6    285347
7    285342
8    285530
9    284989
Name: ticket_id, dtype: int64


<h3>2. Data preparation</h3>
<p>Prepare the datasets X, y, and X_test to resolve the property maintenance fines problem in the city of Detroit, the largest in the state of Michigan, which location in the border between US and Canada.</p>

In [12]:
#Delete and update the X in cases where ticket_issued_dates are nan.
def delete_rows_ticket_issued_date_is_na(df):
    ticket_issued_date = df['ticket_issued_date']
    ticket_issued_date = ticket_issued_date.dropna()
    index_ticket_issued_date = ticket_issued_date.index.tolist()
    ticket_issued_date = ticket_issued_date.reset_index(drop=True)
    df_updated = df[df.index.isin(index_ticket_issued_date)]
    df_updated = df_updated.reset_index(drop=True)
    return df_updated, ticket_issued_date, index_ticket_issued_date

X, ticket_issued_date, index_ticket_issued_date = delete_rows_ticket_issued_date_is_na(X)

#Update y.
y = y[y.index.isin(index_ticket_issued_date)]
y = y.reset_index(drop=True)
#Update X-test
X_test, ticket_issued_date_test, index_ticket_issued_date_test = delete_rows_ticket_issued_date_is_na(X_test)

print(X.shape)
print(y.shape)
print(X_test.shape)

(159880, 27)
(159880,)
(61001, 27)


In [13]:
#Delete rows in a df containing NaN values for the column hearing_date.
#Returns the updated df, and the hearing df column for further analysis.
def delete_rows_hearing_date_is_na(df):
    hearing_date = df['hearing_date']
    hearing_date = hearing_date.dropna()
    index_hearing_date = hearing_date.index.tolist()
    hearing_date = hearing_date.reset_index(drop=True)
    df_updated = df[df.index.isin(index_hearing_date)]
    df_updated = df_updated.reset_index(drop=True)
    return df_updated, hearing_date, index_hearing_date

X, hearing_date, index_hearing_date = delete_rows_hearing_date_is_na(X)

#Update y.
y = y[y.index.isin(index_hearing_date)]
y = y.reset_index(drop=True)

#Update X-test
X_test, hearing_date_test, index_hearing_date_test = delete_rows_hearing_date_is_na(X_test)

print(X.shape)
print(y.shape)
print(X_test.shape)

(159653, 27)
(159653,)
(61001, 27)


In [14]:
#Defining a method that calculates the number of months between the ticket issued date and the hearing date features.
#Add the column diff as the difference in montsh between colummns ticket issued date and hearing date..
#returns the df with the new column.
def diff_months_tiket_issued_and_hearing_date(df):
    ticket_issued_date_times = []
    hearing_date_times = []
    n_months_diff = []
    
    for i in range(0,df.shape[0]):
        a = str(df['ticket_issued_date'].values[i])
        a = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
    
        b = str(df['hearing_date'].values[i])
        b = datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
    
        r = relativedelta.relativedelta(b, a)
        n_months = r.months
        n_months_diff.append(n_months)
        
    n_months_diff = pd.DataFrame(n_months_diff)
    n_months_diff.columns = ['n_months_diff_issued_hearing']

    df_updated = pd.concat([df,n_months_diff], axis =1)
    
    return(df_updated)

#Update X.
X = diff_months_tiket_issued_and_hearing_date(X)

#Calculate the n_months column for the X_test sample.
X_test = diff_months_tiket_issued_and_hearing_date(X_test)

print(X.shape)
print(y.shape)
print(X_test.shape)

(159653, 28)
(159653,)
(61001, 28)


In [20]:
#correct_rows = X_test['n_months_diff_issued_hearing']
#correct_rows = pd.DataFrame(correct_rows)
#correct_rows.columns = ["n_months_diff_issued_hearing"]
#n_months_corrections = []

#for index, row in correct_rows.iterrows():
   # months = row["n_months_diff_issued_hearing"]
   # if(months<0):
    #    months = 30
    #n_months_corrections.append(months)
    
#n_months_corrections = pd.DataFrame(n_months_corrections)  
#n_months_corrections.columns = ["n_months_diff_issued_hearing"]
    
#print(n_months_corrections.head(15))
#print(n_months_corrections.shape)

In [15]:
#Define a method that deletes rows in which the column hearing contains dates happened later than ticket issued dates.
#Deletes rows in which tikets_issued_date is later than hearing_date.
#Returns the df modified without unlogic rows.
def df_correct_issued_date_later_than_hearing_date(df ,is_test_set=False):
    if (is_test_set==False):
        correct_rows = df['n_months_diff_issued_hearing']
        correct_rows = (correct_rows>=0)
        correct_rows = df[correct_rows]
        index_correct_rows = correct_rows.index.tolist()
        df_updated = df[df.index.isin(index_correct_rows)]
        df_updated = df_updated.reset_index(drop=True)
        return df_updated, index_correct_rows
    else:
        correct_rows = df['n_months_diff_issued_hearing']
        correct_rows = pd.DataFrame(correct_rows)
        correct_rows.columns = ["n_months_diff_issued_hearing"]
        n_months_corrections = []

        for index, row in correct_rows.iterrows():
            months = row["n_months_diff_issued_hearing"]
            if(months<0):
                months = 30
            n_months_corrections.append(months)
    
        n_months_corrections = pd.DataFrame(n_months_corrections)  
        n_months_corrections.columns = ["n_months_diff_issued_hearing"]
        
        #Update the dataframe and its column n_months_diff_issued_hearing.
        df_updated = df
        df_updated = df_updated.drop('n_months_diff_issued_hearing',axis=1)
        df_updated = pd.concat([df_updated,n_months_corrections],axis =1)
        
        return df_updated

#Update X.
X, index_correct_rows = df_correct_issued_date_later_than_hearing_date(X, False)

#Update y.
y = y[y.index.isin(index_correct_rows)]
y = y.reset_index(drop=True)

#Update X-test
X_test = df_correct_issued_date_later_than_hearing_date(X_test, True)

print(X.shape)
print(y.shape)
print(X_test.shape)

(159613, 28)
(159613,)
(61001, 28)


In [17]:
#Delete rows having NaN values in the column state for a dataframe passed by parameter.
#Returns the updated df, and the state column for further analysis.
def delete_rows_state_is_na(df):
    state = df['state']
    state = state.dropna()
    index_state = state.index.tolist()
    state = state.reset_index(drop=True)
    df_updated = df[df.index.isin(index_state)]
    df_updated = df_updated.reset_index(drop=True)
    return df_updated, state, index_state

X, state, index_state = delete_rows_state_is_na(X)

#Update y
y = y[y.index.isin(index_state)]
y = y.reset_index(drop=True)

#Update hearing_date
hearing_date = hearing_date[hearing_date.index.isin(index_state)]
hearing_date = hearing_date.reset_index(drop=True)

#X_test
X_test, state_test, index_state_test = delete_rows_state_is_na(X_test)

#hearing_date_test
hearing_date_test = hearing_date_test[hearing_date_test.index.isin(index_state_test)]
hearing_date_test = hearing_date_test.reset_index(drop=True)

print(X.shape)
print(y.shape)
print(X_test.shape)

print("")
print(hearing_date.shape)
print(hearing_date_test.shape)
print(state.shape)
print(state_test.shape)

(159529, 28)
(159529,)
(61001, 28)

(159529,)
(61001,)
(159529,)
(61001,)


In [19]:
#Delete rows in a df containing NaN values for the column violation_code.
#Returns the updated df, and the hearing df column for further analysis.
def delete_rows_violation_code_is_na(df):
    violation_code = df['violation_code']
    violation_code = violation_code.dropna()
    index_violation_code = violation_code.index.tolist()
    violation_code = violation_code.reset_index(drop=True)
    df_updated = df[df.index.isin(index_violation_code)]
    df_updated = df_updated.reset_index(drop=True)
    return df_updated, violation_code, index_violation_code

X, violation_code, index_violation_code = delete_rows_violation_code_is_na(X)

#Update y.
y = y[y.index.isin(index_violation_code)]
y = y.reset_index(drop=True)

#Update X_test.
X_test, violation_code_test, index_violation_code_test = delete_rows_violation_code_is_na(X_test)

#Update hearing_state
hearing_date = hearing_date[hearing_date.index.isin(index_violation_code)]
hearing_date = hearing_date.reset_index(drop=True)

hearing_date_test = hearing_date_test[hearing_date_test.index.isin(index_violation_code)]
hearing_date_test = hearing_date_test.reset_index(drop=True)

#Update state.
state = state[state.index.isin(index_violation_code)]
state = state.reset_index(drop=True)

print(X.shape)
print(y.shape)
print(X_test.shape)

print("")
print(hearing_date.shape)
print(hearing_date_test.shape)
print(state.shape)
print(state_test.shape)
print(violation_code.shape)
print(violation_code_test.shape)

(159529, 28)
(159529,)
(61001, 28)

(159529,)
(61001,)
(159529,)
(61001,)
(159529,)
(61001,)


In [20]:
#Check features containing nan values.
def column_contains_nan(column):
    contains = False
    contains_all = False
    nans = column.isnull().sum()
    if nans > 0:
        contains = True
        if nans == column.shape[0]:
            contains_all = True
    return contains , contains_all

In [21]:
#Droping variables (columns) in df containing only nan values (all rows are nan).
def drop_nan_columns(df):
    df_modified = df
    for i in df_modified.columns:
        contains, contains_all = column_contains_nan(df_modified[i])
        if contains_all==True:
            df_modified = df_modified.drop([i], axis=1)    
    return df_modified

#Update X.
X= drop_nan_columns(X)

#Update X_test.
features = X.columns
X_test = X_test[features]

print(X.dtypes)

print("")
print(X.shape)
print(y.shape)
print(X_test.shape)

ticket_id                         int64
agency_name                      object
inspector_name                   object
violator_name                    object
violation_street_number         float64
violation_street_name            object
mailing_address_str_number      float64
mailing_address_str_name         object
city                             object
state                            object
zip_code                         object
non_us_str_code                  object
country                          object
ticket_issued_date               object
hearing_date                     object
violation_code                   object
violation_description            object
disposition                      object
fine_amount                     float64
admin_fee                       float64
state_fee                       float64
late_fee                        float64
discount_amount                 float64
clean_up_cost                   float64
judgment_amount                 float64


In [22]:
#Define a method that drops from the set of features the variables that aren't relevant in the analysis. 
#A list with the name of irrelevant variables is passed by argument.
def drop_not_relevant_column(df,list_columns):
    df_modified = df
    for i in list_columns:
        df_modified = df_modified.drop([i], axis=1)   
    return df_modified

columns_to_delete = ['violation_street_number','non_us_str_code','mailing_address_str_number']

#Update X.
X = drop_not_relevant_column(X,columns_to_delete)

#Update X_test.
features = X.columns
X_test = X_test[features]

print(X.dtypes)
print("")
print(X.shape)
print(y.shape)
print(X_test.shape)

ticket_id                         int64
agency_name                      object
inspector_name                   object
violator_name                    object
violation_street_name            object
mailing_address_str_name         object
city                             object
state                            object
zip_code                         object
country                          object
ticket_issued_date               object
hearing_date                     object
violation_code                   object
violation_description            object
disposition                      object
fine_amount                     float64
admin_fee                       float64
state_fee                       float64
late_fee                        float64
discount_amount                 float64
clean_up_cost                   float64
judgment_amount                 float64
n_months_diff_issued_hearing      int64
dtype: object

(159529, 23)
(159529,)
(61001, 23)


In [23]:
#Define a method that creates a contingency table with the values in a column that maximixes the compliance rate..
#A column name should be passed by argument.
#The contingency contains the n_top values with more payed tickets in the columns passed by parameter..
#Gives the columns values having the best compliance ratio.
#Returns a df with the ordered values, and a list with their name.
def find_best_ratio_of_payment_column(df_column, df_target, df_column_name, df_target_name, n_top):
  
    df = pd.concat([df_column, df_target], axis =1)
    
    #Create contingency table df.
    df_contingency = pd.crosstab(df[df_column_name], df[df_target_name])
    contingency = df_contingency.values
    
    #Calculate the ratio of best payed tickets regarding on the feature values.
    ratio_best_payed = contingency[:,1]/np.sum(contingency,axis=1)
    ratio_best_payed = pd.DataFrame(ratio_best_payed)
    ratio_best_payed.index = df_contingency.index
    ratio_best_payed.columns = ['ratio_best_payed']
    ratio_best_payed = ratio_best_payed.sort_values(by=['ratio_best_payed'], ascending=False)
    
    #Select the n_top of the contingency ratio df.
    ratio_best_payed = ratio_best_payed.iloc[0:n_top,:]
    list_names = ratio_best_payed.index.tolist()
    
    #Return df_ratio from the contigency with the ordered values.
    return(ratio_best_payed ,list_names)
    
ratio_best_payed_inspector_name, list_names_ratio_best_inspector_name = find_best_ratio_of_payment_column(X['inspector_name'], y, "inspector_name", "compliance", 20)

print(ratio_best_payed_inspector_name.shape)
print(ratio_best_payed_inspector_name.head(5))

print(list_names_ratio_best_inspector_name)
print(len(list_names_ratio_best_inspector_name))

(20, 1)
                   ratio_best_payed
inspector_name                     
Thomas, Duane                   1.0
Coleman, Lanetha                1.0
Hischke, William                1.0
BENNETT, MARGARET               0.5
Long, Phil                      0.4
['Thomas, Duane', 'Coleman, Lanetha', 'Hischke, William', 'BENNETT, MARGARET', 'Long, Phil', 'Pickens, William', 'Frinkley, Elaine', 'Madrigal, Michael', 'Wilcox, Valerie', 'Gibson, Christopher', 'Brinkley, Kevin', 'Crowder, Michael', 'Johnson, Clifford', 'Smith, Melvin', 'Anderson, Trevis', 'Shah , Kumarpal', 'Copty, Anton', 'Zawislak, Norbet', 'Holbrook, Kevin', 'Fulks, Matthew g']
20


In [24]:
#Define a method that creates a df having the contingency values for the n_top most frequent values.
#Returns a df with the n_top values having more ticktes
def most_frequent_values_in_feature(df_column, df_y,  n_top):
    
    df_contingency_1 = pd.crosstab(df_column, y)
    df_contingency = df_contingency_1.values
    n_tickets = np.sum(df_contingency,axis=1)
    n_tickets = pd.DataFrame(n_tickets)
    n_tickets.index = df_contingency_1.index
    n_tickets.columns = ['n_tickets']
    n_tickets = n_tickets.sort_values(by=['n_tickets'], ascending=False)
    n_tickets = n_tickets.iloc[0:n_top,:]
    list_names = n_tickets.index.tolist()
    
    #Return df_ratio from the contigency with the ordered values.
    return(n_tickets ,list_names)

df_frequents_inspector_name, list_names_inspector_name = most_frequent_values_in_feature(X['inspector_name'], y , 5)

print(df_frequents_inspector_name)
print(len(list_names_inspector_name))

                  n_tickets
inspector_name             
Morris, John          11604
Samaan, Neil J         8699
O'Neal, Claude         8067
Steele, Jonathan       6957
Devaney, John          6837
5


In [25]:
#Define a method that encodes dataframe categorical variables.
#A list of columns to encode, and the name of the column must be passed by argument.
#Returns the updated df with the encodings.
def encode_dummy_given_list(df, list_names, column_name):    
    df_update = df
    for i in list_names:
        column = (df_update[column_name]==i).astype(int)
        column = pd.DataFrame(column)
        column.columns = [column_name+"_"+i]
        df_update = pd.concat([df_update,column], axis =1)
        
    df_update = df_update.drop(column_name, axis=1)
    
    return df_update

X = encode_dummy_given_list(X,list_names_inspector_name,"inspector_name")

#Update X_test
X_test = encode_dummy_given_list(X_test,list_names_inspector_name,"inspector_name")

print(X.shape)
print(y.shape)
print(X_test.shape)

(159529, 27)
(159529,)
(61001, 27)


In [26]:
#Calculate best ratio payments for violation code.
ratio_best_payed_violation_code, list_names_ratio_best_violation_code = find_best_ratio_of_payment_column(violation_code, y, "violation_code", "compliance", 5)

print(ratio_best_payed_violation_code.shape)
print(ratio_best_payed_violation_code.head(5))
print("")
print(list_names_ratio_best_violation_code)
print(len(list_names_ratio_best_violation_code))

(5, 1)
                      ratio_best_payed
violation_code                        
61-47.0000/47.0108                 1.0
9-1-83 - (Building 5               0.5
19840901                           0.5
22-2-87(a)                         0.5
9-1-333                            0.5

['61-47.0000/47.0108', '9-1-83 - (Building 5', '19840901', '22-2-87(a)', '9-1-333']
5


In [27]:
#Calculate the most frequent values for the column violation_code.
df_frequents_violation_code, list_names_violation_code = most_frequent_values_in_feature(violation_code, y , 5)

print(df_frequents_violation_code)
print(len(list_names_violation_code))

                n_tickets
violation_code           
9-1-36(a)           64335
9-1-81(a)           23133
22-2-88             18989
9-1-104             16871
22-2-88(b)           4821
5


In [28]:
#Encode values of violation_code found with the best violation_code..
#Update X.
X = encode_dummy_given_list(X, list_names_violation_code, "violation_code")

#Update X_test.
X_test = encode_dummy_given_list(X_test, list_names_violation_code, "violation_code")

print(X.shape)
print(y.shape)
print(X_test.shape)

(159529, 31)
(159529,)
(61001, 31)


In [29]:
#Calculate best ratio payments for the column state.
ratio_best_payed_state, list_names_ratio_best_state = find_best_ratio_of_payment_column(state, y, "state", "compliance", 20)

print(ratio_best_payed_state.shape)
print(ratio_best_payed_state.head(5))
print("")
print(list_names_ratio_best_state)
print(len(list_names_ratio_best_state))

(20, 1)
       ratio_best_payed
state                  
QL             1.000000
HI             0.333333
RI             0.266667
KS             0.230769
NH             0.222222

['QL', 'HI', 'RI', 'KS', 'NH', 'NB', 'KY', 'CO', 'SD', 'OH', 'IL', 'MD', 'AK', 'WY', 'WA', 'NY', 'MI', 'NJ', 'WI', 'ON']
20


In [30]:
#Calculate the most frequent values for the column state.
df_frequents_state, list_names_state = most_frequent_values_in_feature(state, y , 5)

print(df_frequents_state)
print(len(list_names_state))

       n_tickets
state           
MI        143415
CA          3823
TX          1946
FL          1679
SC          1066
5


In [31]:
#Encode values of violation_code found with the best state ratio.
#Update X.
X = encode_dummy_given_list(X, list_names_state, "state")

#Update X_test.
X_test = encode_dummy_given_list(X_test, list_names_state, "state")

print(X.shape)
print(y.shape)
print(X_test.shape)

(159529, 35)
(159529,)
(61001, 35)


In [32]:
#Droping variables (columns) having too many categories.
def drop_too_many_categories_in_column(df):
    df_modified = df
    for i in df_modified.columns:
        if(i != 'ticket_id'):
            if(df_modified.dtypes[i]=='object'):
                n_classes = df_modified[i].nunique()
                if n_classes >10:
                    df_modified = df_modified.drop([i], axis=1)   
    return df_modified

#Update X.
X = drop_too_many_categories_in_column(X)

#Update X_test.
features = X.columns
X_test = X_test[features]

print(X.dtypes)

print("")
print(X.shape)
print(y.shape)
print(X_test.shape)

ticket_id                            int64
agency_name                         object
country                             object
disposition                         object
fine_amount                        float64
admin_fee                          float64
state_fee                          float64
late_fee                           float64
discount_amount                    float64
clean_up_cost                      float64
judgment_amount                    float64
n_months_diff_issued_hearing         int64
inspector_name_Morris, John          int64
inspector_name_Samaan, Neil J        int64
inspector_name_O'Neal, Claude        int64
inspector_name_Steele, Jonathan      int64
inspector_name_Devaney, John         int64
violation_code_9-1-36(a)             int64
violation_code_9-1-81(a)             int64
violation_code_22-2-88               int64
violation_code_9-1-104               int64
violation_code_22-2-88(b)            int64
state_MI                             int64
state_CA   

In [33]:
#Extract from the hearing date column the status and month for further analysis.
#Two additional features are created.
def hearing_date_feature_extraction(df_hearing_date):
    
    #tranform the type of the column hearing_date into date. 
    hearing_date = pd.to_datetime(df_hearing_date)
    
    #Extract the years column from hearing_date.
    hearing_date_years = hearing_date.dt.year
    hearing_date_years = pd.DataFrame(hearing_date_years)
    hearing_date_years.rename(columns = {'hearing_date':'hearing_date_year'}, inplace = True)
    hearing_date_years = hearing_date_years.astype('float64')
    
    #Extract the months column from hearing_date.
    hearing_date_months = hearing_date.dt.month
    hearing_date_months = pd.DataFrame(hearing_date_months)
    hearing_date_months.rename(columns = {'hearing_date':'hearing_date_month'}, inplace = True)
    hearing_date_months = hearing_date_months.astype('float64')
    
    #Extract the column hearing_date_status from the df.
    conditions = [
        (hearing_date_years['hearing_date_year'] <= 2011),
        (hearing_date_years['hearing_date_year'] > 2011) & (hearing_date_years['hearing_date_year'] <= 2015),
        (hearing_date_years['hearing_date_year'] > 2015)
    ]

    # create a list of the values we want to assign for each condition
    values = ['old', 'partially-old', 'recent']
    # create a new column and use np.select to assign values to it using our lists as arguments
    hearing_date_status = pd.DataFrame(np.select(conditions, values))
    hearing_date_status.rename(columns = {0: 'hearing_date_status'}, inplace = True)
    
    #Reset the indexes of the created dataframes.
    hearing_date_status = hearing_date_status.reset_index(drop=True)
    hearing_date_months = hearing_date_months.reset_index(drop=True)
    
    #Change the types of the hearing_date_months column.
    hearing_date_months = pd.DataFrame(hearing_date_months)
    hearing_date_months = hearing_date_months.astype(str)
    
    #return the extracted columns.
    return hearing_date_status, hearing_date_months

#Extract the new variables from the feauture X set.
hearing_date_status, hearing_date_months = hearing_date_feature_extraction(hearing_date)

#Extract the new variables from the feauture X_test set.
hearing_date_status_test, hearing_date_months_test = hearing_date_feature_extraction(hearing_date_test)


print("")
print(X.shape)
print(y.shape)
print(X_test.shape)

print("")
print(hearing_date_status.shape)
print(hearing_date_months.shape)
print("")
print(hearing_date_status_test.shape)
print(hearing_date_months_test.shape)


(159529, 27)
(159529,)
(61001, 27)

(159529, 1)
(159529, 1)

(61001, 1)
(61001, 1)


<h3>2. Engineering Data Analysis (EDA)</h3>
<p>Check the relevance of some categorical features in relationship with the target variable.The analysis focuses over some extracted variables during the preparation phase.</p>
<ul>
<li>The variables of interest are n_months_difference, hearing_status, state, inspector_name, and violation_code.</li>
</ul>

In [34]:
#Check the changes in the target variable in regards to the variable n_months_difference. 
#df_diff_months = pd.concat([X['n_months_diff_issued_hearing'], y], axis =1)

#sns.countplot(data=df_diff_months, x='n_months_diff_issued_hearing',hue = 'compliance')
#plt.show()

#print("contingency_table")
#print(pd.crosstab(df_diff_months["n_months_diff_issued_hearing"], df_diff_months["compliance"]))

#print("")
#print(X['n_months_diff_issued_hearing'].unique())
#print(X.groupby('n_months_diff_issued_hearing').size())

In [35]:
#Analyse whether the column hearing_date_status is relevant for the target.
#Countplots are created.
#df_status = pd.concat([y, hearing_date_status], axis =1)
#print(df_status.shape)
#sns.countplot(data=df_status, x='hearing_date_status',hue = 'compliance')
#plt.show()

#print("contingency_table")
#print(pd.crosstab(df_status["hearing_date_status"], df_status["compliance"]))

In [36]:
#Analyse whether the column violation_code is relevant for the target.
#Countplots are created.
#df_violation_code = pd.concat([violation_code, y], axis =1)
#print(df_violation_code.shape)
#sns.countplot(data=df_violation_code, x='violation_code',hue = 'compliance')
#plt.show()
#print(df_violation_code.shape)
#print("")
#print("contingency_table")
#print(pd.crosstab(df_violation_code["violation_code"], df_violation_code["compliance"]))
#The most relevant are 19450901, 22-2-17, 22-2-43, 22-2-45, 9-1-81(a).s

In [37]:
#Analyse whether the column hearing_dates_months is relevant for the target.
#Countplots are created.
#df_month = pd.concat([hearing_date_months,y], axis =1)
#print(df_month.shape)
#sns.countplot(data=df_month, x='hearing_date_month',hue = 'compliance')
#plt.show()

#print("contingency_table")
#print(pd.crosstab(df_month["hearing_date_month"], df_month["compliance"]))

In [38]:
#Check the balance of classes in the target variable.
#The relationship is almost 1 to 9 for the classes in the target variable.
y_classes = pd.DataFrame(y)
y_classes.columns = ['values']
print(y_classes['values'].value_counts(normalize=True))

0.0    0.928421
1.0    0.071579
Name: values, dtype: float64


<h3>3. Dataset encoding</h3>
<p>Transforming the dateset into all numerical entries. Features containing categorical values are converted into dummy vectors. It means that each category is represented by a vector of ones (1) and zeros (0).</p>
<ul>
<li>The procedure is done over the whole dataset because features on the train, and test sets might be equal.</li>
</ul>

In [39]:
#Define the number of rows in the X matrix.
n_X = X.shape[0]

#Concatenate vertically X, and X_test matrixes and encode them together.
concatenated_dataframes = pd.concat([X, X_test], axis=0)
concatenated_dataframes = pd.get_dummies(concatenated_dataframes)

#Split the X and X_test matrixes after been encoded.
X = concatenated_dataframes.iloc[0:n_X,:]
X_test = concatenated_dataframes.iloc[n_X:,:]

#Reshaping the y matrix into a vector (n,).
y = y.values.reshape(y.values.shape[0],)
print(y.shape)

print(X.dtypes)
print("")
print(X.shape)
print(y.shape)
print(X_test.shape)

(159529,)
ticket_id                                                       int64
fine_amount                                                   float64
admin_fee                                                     float64
state_fee                                                     float64
late_fee                                                      float64
discount_amount                                               float64
clean_up_cost                                                 float64
judgment_amount                                               float64
n_months_diff_issued_hearing                                    int64
inspector_name_Morris, John                                     int64
inspector_name_Samaan, Neil J                                   int64
inspector_name_O'Neal, Claude                                   int64
inspector_name_Steele, Jonathan                                 int64
inspector_name_Devaney, John                                    int64
violation_

<h3>4. Feature selection (dimensionality reduction)</h3>
<p>Select features having the strongest relationship with the target variable. Dimensionality reduction guarantees a best performance during the training phase</p>
<ul>
<li>In order to select the features, information gain is applied. It implies that features are selected satisfying the smallest lost of explained information when removing the variables</li>
<li>Information gain is done by minimizing the entropy of the matrix X once variables are put taken off/li>
</ul>

In [40]:
#Define a method that performs feature selection on the set of features.
#Select the n_top feauture variables from the dataset X which better explain y regarding on information gain.
#Information gain is based on entropy minimization when selection features.
#Returns a list with the names of the selected n_features.
def feature_selection_information_gain(X, y, n_top_features):
    mutual_info = mutual_info_classif(X,y)
    mutual_info = pd.DataFrame(mutual_info)
    mutual_info.index = X.columns
    mutual_info.columns = ['values']
    mutual_info.sort_values(by='values', ascending=False, inplace =True)
    
    df_relevant_features = mutual_info
    df_relevant_features = df_relevant_features.iloc[0:n_top_features,:]
    selected_features = df_relevant_features.index.tolist()
    print(df_relevant_features)
    return(selected_features)

selected_features = feature_selection_information_gain(X, y, 15)

#Select the features according to their importance
X = X[selected_features]
X_test = X_test[selected_features]

print(selected_features)
print(X.shape)
print(X_test.shape)

                                                      values
disposition_Responsible by Default                  0.064177
judgment_amount                                     0.052145
late_fee                                            0.048161
state_fee                                           0.036437
admin_fee                                           0.036093
country_USA                                         0.035244
state_MI                                            0.026306
ticket_id                                           0.023753
disposition_Responsible by Admission                0.018638
discount_amount                                     0.015319
agency_name_Buildings, Safety Engineering & Env...  0.012201
disposition_Responsible by Determination            0.012159
fine_amount                                         0.011938
violation_code_9-1-36(a)                            0.005580
n_months_diff_issued_hearing                        0.004862
['disposition_Responsibl

In [41]:
#Select the best K features.
def feature_selection_information_gain_KBest(X, y, n_top):
    sel_cols = SelectKBest(mutual_info_classif, k = n_top)
    sel_cols.fit(X,y)
    selected_features = X.columns[sel_cols.get_support()].tolist()
    return(selected_features)

selected_features_KBest = feature_selection_information_gain_KBest(X, y, 15)

print(selected_features_KBest)

['disposition_Responsible by Default', 'judgment_amount', 'late_fee', 'state_fee', 'admin_fee', 'country_USA', 'state_MI', 'ticket_id', 'disposition_Responsible by Admission', 'discount_amount', 'agency_name_Buildings, Safety Engineering & Env Department', 'disposition_Responsible by Determination', 'fine_amount', 'violation_code_9-1-36(a)', 'n_months_diff_issued_hearing']


<h3>5. Training phase and model selection</h3>
<p>Training multiple classifiers on the X matrix. After models are trained, the one having the best performance is selected.</p>
<ul>
<li>Hyperparameter tuning is done over each model. No more than 30 mins of training per model is satisfied</li>
<li>Classifiers like SVM, Logistic regression, Random forest, and Gradient boosting are used for the purpose of this phase.</li>
</ul>

In [42]:
#Define a method that returns a series object with the ids and probabilities of compliance.
def get_probs_format(X, y_prob):
    ticket_predictions = pd.DataFrame(y_prob)
    ticket_predictions.index = X["ticket_id"]
    ticket_predictions = ticket_predictions.iloc[:,1:2]
    ticket_predictions.columns = ["compliance"]
    ticket_predictions = ticket_predictions.squeeze()
    return ticket_predictions

In [ ]:
#n_months_diff_issued_hearing
print(X_test['n_months_diff_issued_hearing'].unique())

In [46]:
#Train a logistic regresion classifier.
#Obtained AUC in test = 0.775

#Best AUC reached : 0.77909119. running best 20 and selectiog 30.
#Best AUC 0.78433477 running best 5 and selecting 18.
#Best AUC 0.77989349 running best 5 and selecting 10.
#Best AUC 0.78518753 running best 10 and selecting 25.

scaler = MinMaxScaler()
clf = LogisticRegression()
pipeline = Pipeline([('scaler',scaler), ('model', clf)])
grid_values = {'model__penalty': ['l1', 'l2'], 'model__C':[0.01, 0.1, 1, 10, 100]}
search = GridSearchCV(pipeline, grid_values, cv =5, scoring = "roc_auc").fit(X,y)

cv_result = search.cv_results_

mean_train_score = cv_result['mean_train_score']

mean_test_score = cv_result['mean_test_score']

results_train = np.array(mean_train_score).reshape(5,2)

results = np.array(mean_test_score).reshape(5,2)

params = search.best_params_

#For each combination of parameters, there is an average of the auc metric calculated within a logistic regression classifier.
#L1,l2 as columns, and 0.01, 0.1, 0.1, 10, 100 as rows in the results matrix.
print(cv_result)
print('')
print('train')
print(results_train)
print('')
print('test')
print(results)
print('')
print(params)
print('')

#Predict probabilities.
y_prob_logistic_regression = search.predict_proba(X)

print(y_prob_logistic_regression)

#Get probabilities.
y_prob_logistic_regression = get_probs_format(X,y_prob_logistic_regression)
print(y_prob_logistic_regression)

{'split0_test_score': array([ 0.7296437 ,  0.7300794 ,  0.75147205,  0.73593671,  0.75538164,
        0.74791049,  0.75655579,  0.75491986,  0.75668577,  0.75624926]), 'split1_test_score': array([ 0.72286973,  0.74555479,  0.77688246,  0.7587999 ,  0.78194468,
        0.7753049 ,  0.78211259,  0.78120291,  0.78213495,  0.78200429]), 'split2_test_score': array([ 0.78209667,  0.78901652,  0.82300475,  0.80588357,  0.83214228,
        0.82134168,  0.8330905 ,  0.83049403,  0.83317444,  0.83283282]), 'split3_test_score': array([ 0.73439603,  0.73414522,  0.76750971,  0.74542214,  0.76893614,
        0.76139743,  0.76912173,  0.77012506,  0.76872657,  0.76907289]), 'split4_test_score': array([ 0.75844167,  0.74597382,  0.76116849,  0.75503033,  0.76206051,
        0.76213718,  0.76112213,  0.76576007,  0.76095419,  0.76556319]), 'mean_test_score': array([ 0.74548948,  0.74895397,  0.77600758,  0.76021456,  0.78009316,
        0.77361841,  0.78040067,  0.78050048,  0.78033531,  0.78114459]),

In [48]:
#Train a randomforest classifier with randomsearch.
#Obtained AUC in test = 

#The classifier is tried with randomsearch to check the best parameters combinations that make better classifications.
#Crossvalidation is not performed with this classifier cause it is nessesary an X_test(validation) to evaluate the auc metric.
#X = X[selected_features]
#X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.20)

#The parameter class_weight='balanced_subsample' will check the proportion of labels of each class in each tree.
#Based on that, it will calculate the metric in favor to the minority class.
#clf = RandomForestClassifier(class_weight='balanced_subsample')

#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#n_estimators = [200]
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 210, num = 2)]
# Number of features to consider at every split
#max_features = ['auto']
#max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
#max_depth = [10]
#max_depth = [int(x) for x in np.linspace(10, 110, num = 10)]
#max_depth = [int(x) for x in np.linspace(10, 20, num = 2)]
#max_depth.append(None)
#min_samples_split[2]
# Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
#min_samples_leaf = [1]
#min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#bootstrap = [True]
#bootstrap = [True, False]

#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#n_iter tells us the number of different combinations to try.
#rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 5, cv = 5, verbose=2, random_state=42, n_jobs = -1, scoring ="roc_auc")# Fit the random search model

#Training the classifier.
#rf_random.fit(X_train, y_train)

#Print results
#auc_train = roc_auc_score(y_train, rf_random.predict(X_train))
#auc_test = roc_auc_score(y_val, rf_random.predict(X_val))

#Predict probabilities.
#y_prob_random_forest_rs = grid_search.predict_proba(X_val)

#print(rf_random.best_params_)
#print(auc_train)
#print(auc_test)

#print(y_prob_random_forest_rs)

In [49]:
#Train a randomforest classifier with gridsearch.
#Obtained AUC in test = 0.66

#Try a grid search classifier with gridsearch to see parameters combinations that make better classifications.
#Crossvalidation is not performed with this classifier cause it is nessesary an X_test(validation) to evaluate the auc metric.
#from sklearn.model_selection import GridSearchCV, KFold

#X = X[selected_features]
#X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.20)

#The parameter class_weight='balanced_subsample' will check the proportion of labels of each class in each tree.
#Based on that, it will calculate the metric in favor to the minority class.
#clf = RandomForestClassifier(class_weight='balanced_subsample')

#param_grid = {
#    'bootstrap': [True],
#    #'max_depth': [80, 90, 100, 110],
#    'max_features': ['auto', 'sqrt'],
#    'min_samples_leaf': [5,6],
#    #'min_samples_split': [8, 10, 12],
#    'n_estimators': [50, 75, 150]
#}# Create a based model

#grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

#Training the classifier.
#grid_search.fit(X_train, y_train)

#Print results
#auc_train = roc_auc_score(y_train, grid_search.predict(X_train))
#auc_test = roc_auc_score(y_val, grid_search.predict(X_val))

#Predict probabilities.
#y_prob_random_forest_rs = grid_search.predict_proba(X_val)

#print(grid_search.best_params_)
#print(auc_train)
#print(auc_test)

#print("")
#print(y_prob_random_forest_rs)

In [50]:
#Train a supporting vector machines classifier.
#Obtained AUC in test = 

#from sklearn.model_selection import GridSearchCV, KFold

#X = X[selected_features]
#X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.20)

#The parameter class_weight='balanced_subsample' will check the proportion of labels of each class in each tree.
#Based on that, it will calculate the metric in favor to the minority class.
#clf = SVC(class_weight='balanced', probability=True)

#param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              #'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              #'kernel': ['rbf']} 
        
#param_grid = {'C': [1], 
  #            'gamma': [1],
  #            'kernel': ['rbf']}

#grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, 
                         # cv = 3, n_jobs = -1, verbose = 2)

#Training the classifier.
#grid_search.fit(X_train, y_train)

#Print results
#auc_train = roc_auc_score(y_train, grid_search.predict(X_train))
#auc_test = roc_auc_score(y_val, grid_search.predict(X_val))

#Predict probabilities
#y_prob_SVM = grid_search.predict_proba(X_val)

#print(grid_search.best_params_)
#print(auc_train)
#print(auc_test)

#print("")
#print(y_prob_SVM)

In [51]:
#Try a gradient boosting classifier on the problem.
#Obtained AUC in test = 0.623612649317

#X = X[selected_features]
#X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.20)

#The parameter class_weight='balanced_subsample' will check the proportion of labels of each class in each tree.
#Based on that, it will calculate the metric in favor to the minority class.
#clf = GradientBoostingClassifier()

#Finding the optimal number of desicion trees.
#errors = [mean_squared_error(y_test, y_pred) for y_pred in regressor.staged_predict(X_test)]
#best_n_estimators = np.argmin(errors)

#parameters = {
#    "learning_rate": [0.01,0.05, 0.1],
#    "n_estimators":[10,50,100],
#    "subsample":[0.5, 0.8, 0.85]
#}

#parameters = {
    #"loss":["deviance"],
    #"learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
    #"max_depth":[3,5,8],
    #"max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
    #"subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #"n_estimators":[10]
#}


#grid_search = GridSearchCV(estimator = clf, param_grid = parameters, scoring = "roc_auc", cv = 5, n_jobs = -1, verbose = 2)

#Training the classifier.
#grid_search.fit(X_train, y_train)

#Print results
#auc_train = roc_auc_score(y_train, grid_search.predict(X_train))
#auc_test = roc_auc_score(y_val, grid_search.predict(X_val))

#print(grid_search.best_params_)
#print(auc_train)
#print(auc_test)

#Predict probabilities
#y_prob_GB = grid_search.predict_proba(X_val)

#print(y_prob_GB)

<h3>6. Model evaluation on unseen data </h3>
<p>Evaluating how well the model performs on data that it hasn't seen before. The evaluation is performed over the X_test set.</p>
<ul>
<li>A Logistic Regression was selected as the best classifier to perform the prediction task for ticket compliance during the training and model selection phase </li>
<li>At the final part, probabilites of the given predictions are calculated.</li>
</ul>

In [52]:
#Logistic regression prediction over the test set.
#Predict probabilities.
y_prob_logistic_regression = search.predict_proba(X_test)
y_prob_logistic_regression = get_probs_format(X_test, y_prob_logistic_regression)
print(y_prob_logistic_regression)
print("")
print(y_prob_logistic_regression.shape)

ticket_id
284932    0.057680
285362    0.020298
285361    0.062121
285338    0.055008
285346    0.062120
285345    0.055009
285347    0.065990
285342    0.481184
285530    0.018669
284989    0.036749
285344    0.065990
285343    0.020297
285340    0.020297
285341    0.065990
285349    0.057326
285348    0.050734
284991    0.036749
285532    0.038018
285406    0.038014
285001    0.035770
285006    0.010449
285405    0.018667
285337    0.036759
285496    0.069186
285497    0.057706
285378    0.019617
285589    0.035016
285585    0.055019
285501    0.065143
285581    0.020301
            ...   
376367    0.014764
376366    0.050012
376362    0.348011
376363    0.236119
376365    0.014764
376364    0.050012
376228    0.052465
376265    0.052467
376286    0.293951
376320    0.050010
376314    0.050010
376327    0.283567
376385    0.283577
376435    0.465377
376370    0.496949
376434    0.074343
376459    0.070021
376478    0.000134
376473    0.052476
376484    0.027414
376482    0.019597
37

In [53]:
def blight_model():
    
    answer = y_prob_logistic_regression
    
    return answer

In [54]:
blight_model()

ticket_id
284932    0.057680
285362    0.020298
285361    0.062121
285338    0.055008
285346    0.062120
285345    0.055009
285347    0.065990
285342    0.481184
285530    0.018669
284989    0.036749
285344    0.065990
285343    0.020297
285340    0.020297
285341    0.065990
285349    0.057326
285348    0.050734
284991    0.036749
285532    0.038018
285406    0.038014
285001    0.035770
285006    0.010449
285405    0.018667
285337    0.036759
285496    0.069186
285497    0.057706
285378    0.019617
285589    0.035016
285585    0.055019
285501    0.065143
285581    0.020301
            ...   
376367    0.014764
376366    0.050012
376362    0.348011
376363    0.236119
376365    0.014764
376364    0.050012
376228    0.052465
376265    0.052467
376286    0.293951
376320    0.050010
376314    0.050010
376327    0.283567
376385    0.283577
376435    0.465377
376370    0.496949
376434    0.074343
376459    0.070021
376478    0.000134
376473    0.052476
376484    0.027414
376482    0.019597
37